In [ ]:
# Imports
# pip install kagglehub[pandas-datasets]
#import kagglehub
#from kagglehub import KaggleDatasetAdapter
import os
import random
from glob import glob
import cv2 
import matplotlib.pylab as plt
import seaborn as sns
import pandas as pd
import numpy as np


import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

import argparse
#import torch
#import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Data over

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx].astype(np.uint8)
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

# Load the data(images)
x_npz = np.load("../../data/x_images_arrays.npz")
X = x_npz["arr_0"]
y_npz = np.load("../../data/y_labels_arrays.npz")
Y = y_npz["arr_0"]

print(np.array(X).shape)
print(np.array(Y).shape)

# Separete the data into train and test sets
# 80% train, 20% test
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=1, stratify=Y)

#x_test, x_val, y_test, y_val = train_test_split(x_temp, y_temp, test_size=0.5, random_state=1, stratify=y_temp)

print(np.array(x_train).shape)
#print(np.array(x_val).shape)
print(np.array(x_test).shape)
# (3045, 128, 128, 3)
# (381, 128, 128, 3)
# (381, 128, 128, 3)

# Transform the data to tensor
# Define a transform
transform = transforms.Compose([
            #transforms.Resize((28, 28)),
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize((0,), (1,))])

# Apply the transform to the data
dataset_train = CustomImageDataset(x_train, y_train, transform=transform)
#dataset_val = CustomImageDataset(x_val, y_val, transform=transform)
dataset_test = CustomImageDataset(x_test, y_test, transform=transform)

img, label = dataset_train[0]
print(img.shape, label) # [grayscale=1, size=128, size=128] label=0 ('Nothing')
print(dataset_train.__len__()) # 3045 images
#print(dataset_val.__len__()) # 381 images
print(dataset_test.__len__()) # 381 images

# Load into the DataLoader
batch_size = 32

train_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
#val_loader = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


In [ ]:
# Oversampling
import numpy as np
from oversampling.o2pf import O2PF
from common.common import COMMON
import sys

if len(sys.argv) <= 2:
	print('Usage: %s <train dataset>  <k_max>' % sys.argv[0])
	raise SystemExit

train = np.loadtxt(sys.argv[1],delimiter=',', dtype=np.float32)

X = train[:,:-1]
y = train[:,-1].astype(int)

o2pf = O2PF(k_max=int(sys.argv[2]))
X_res, y_res = o2pf.fit_resample(X, y)

path = 'data'
common = COMMON()
common.saveDataset(X_res, y_res, path, o2pf.__class__.__name__)

In [ ]:
#undersampling
import numpy as np
from undersampling.opf_us import OpfUS
from common.common import COMMON
import sys

if len(sys.argv) <= 1:
	print('Usage: %s <train dataset> <validation dataset (optional)>' % sys.argv[0])
	raise SystemExit

train = np.loadtxt(sys.argv[1],delimiter=',', dtype=np.float32)

valid = None
if len(sys.argv)>=3:
	valid = np.loadtxt(sys.argv[2],delimiter=',', dtype=np.float32)		
	concat = np.concatenate((train, valid))
	X = concat[:,:-1]
	y = concat[:,-1].astype(np.int) 
else:
	X = train[:,:-1]
	y = train[:,-1].astype(int)

opf_us = OpfUS()
X_res, y_res = opf_us.fit_resample(X, y, valid)

path = 'data'
common = COMMON()
common.saveDataset(X_res, y_res, path, opf_us.__class__.__name__)